In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import os
import itertools
import warnings
import glob
import sys

In [2]:
staff = pd.read_csv("people_flt.csv")
staff.shape[0]

124

In [3]:
staff_flt = staff[(~staff["MTMT"].isnull()) & (staff["MTMT"]!=11111111) & (staff["MTMT"]!=0) & (staff["Munkakör"].str.lower()!="doktorandusz") & (staff["Munkakör"].str.lower()!="sh doktorandusz")]
staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")
staff_flt.shape[0]

/tmp/ipykernel_492721/3697786990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["MTMT"] = staff_flt["MTMT"].astype("int")


119

In [4]:
def load_pub_for_author(mtid):
    page = 1
    size = 1000
    tries = 1
    author_pubs = []
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=published;eq;true&cond=core;eq;true&cond=authors.mtid;eq;{mtid}&sort=publishedYear,desc&sort=firstAuthor,asc&size={size}&fields=citations,pubStats&labelLang=hun&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if page == 1 and len(dta["content"]) == 0:
                    return []
                author_pubs += dta["content"]
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
    return author_pubs

In [5]:
for mtid in tqdm(staff_flt["MTMT"]):
    if not os.path.exists(f"personal/{mtid}.json"):
        json.dump(load_pub_for_author(mtid), open(f"personal/{mtid}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)

 76%|███████████████████████████████▊          | 90/119 [12:48<03:27,  7.16s/it]

Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying


 76%|████████████████████████████████          | 91/119 [15:20<23:33, 50.49s/it]

Error, retrying
Error, retrying


 87%|███████████████████████████████████▍     | 103/119 [17:23<02:01,  7.58s/it]

Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying
Error, retrying


100%|█████████████████████████████████████████| 119/119 [22:00<00:00, 11.10s/it]


In [6]:
def get_h_score(mtid):
    cit_list = []
    if os.path.exists(f'personal/{mtid}.json'):
        with open(f'personal/{mtid}.json') as f:
            try:
                js = json.load(f)
                for pub in js:
                    if "independentCitationCount" in pub:
                        cit_list.append(pub["independentCitationCount"])
            except:
                pass
    cit_list.sort(reverse=True)
    hindex = 0
    while hindex < len(cit_list) and hindex < cit_list[hindex]:
        hindex += 1
    return hindex

In [7]:
staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]

/tmp/ipykernel_492721/2996616290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_flt["H index"] = [0 if np.isnan(mtid) else get_h_score(int(mtid)) for mtid in staff_flt["MTMT"]]


In [8]:
esi_cats = pd.read_csv("esi-master-journal-list-8-2022.csv")
percentiles = pd.read_csv("baselinecitationpercentiles2022.csv")
percentiles.set_index(keys=["category","year"], inplace=True)

def get_esi_category(jour):
    with open(f'journals/{jour}.json') as fj:
        data = json.load(fj)["content"]
        cat = []
        if "abbreviated" in data:
            cat = esi_cats[esi_cats["Title29"]==data["abbreviated"]]["Category name"]
            if len(cat) == 0:
                cat = esi_cats[esi_cats["Title20"]==data["abbreviated"]]["Category name"]
    return cat.iloc[0]
def check_wos(pub):
    if "journal" in pub and "sciIndexed" in pub["journal"] and pub["journal"]["sciIndexed"]:
        return True
    if "identifiers" in pub:
        for i in pub["identifiers"]:
            if i["source"]["label"] == "WoS" or i["source"]["label"] == "Wos-CSCD (Chinese)":
                return True
    return False

In [9]:
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'personal/{mtid}.json'):
            with open(f'personal/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if not os.path.exists(f'journals/{jour}.json'):
                                print("Új folyóirat: ", pub["journal"]["label"])
                                os.system(f'wget -O journals/{jour}.json "https://m2.mtmt.hu/api/journal/{jour}"')
                except:
                    pass
        else:
            print("Hiányzó szerző!", mtid)

0it [00:00, ?it/s]--2023-01-24 15:57:31--  https://m2.mtmt.hu/api/journal/9962
Resolving m2.mtmt.hu (m2.mtmt.hu)... 

Új folyóirat:  REVIEWS IN MATHEMATICAL PHYSICS 0129-055X 1793-6659


193.224.44.74
Connecting to m2.mtmt.hu (m2.mtmt.hu)|193.224.44.74|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/vnd.mtmt2-1.0+json]
Saving to: ‘journals/9962.json’

     0K ...                                                    40,8K=0,1s

2023-01-24 15:57:33 (40,8 KB/s) - ‘journals/9962.json’ saved [4065]

--2023-01-24 15:57:33--  https://m2.mtmt.hu/api/journal/10005881
Resolving m2.mtmt.hu (m2.mtmt.hu)... 

Új folyóirat:  JOURNAL OF PHYSICS A-MATHEMATICAL AND THEORETICAL 1751-8113 1751-8121


193.224.44.74
Connecting to m2.mtmt.hu (m2.mtmt.hu)|193.224.44.74|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/vnd.mtmt2-1.0+json]
Saving to: ‘journals/10005881.json’

     0K ......                                                 68,4K=0,1s

2023-01-24 15:57:34 (68,4 KB/s) - ‘journals/10005881.json’ saved [7137]

--2023-01-24 15:57:34--  https://m2.mtmt.hu/api/journal/10005675
Resolving m2.mtmt.hu (m2.mtmt.hu)... 

...

connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/vnd.mtmt2-1.0+json]
Saving to: ‘journals/3180.json’

     0K ...                                                    39,6K=0,1s

2023-01-24 16:06:38 (39,6 KB/s) - ‘journals/3180.json’ saved [4082]

--2023-01-24 16:06:38--  https://m2.mtmt.hu/api/journal/3242
Resolving m2.mtmt.hu (m2.mtmt.hu)... 

Új folyóirat:  MODERN PHYSICS LETTERS A 0217-7323 1793-6632


193.224.44.74
Connecting to m2.mtmt.hu (m2.mtmt.hu)|193.224.44.74|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/vnd.mtmt2-1.0+json]
Saving to: ‘journals/3242.json’

     0K ....                                                   16,7K=0,3s

2023-01-24 16:06:39 (16,7 KB/s) - ‘journals/3242.json’ saved [4493]

--2023-01-24 16:06:40--  https://m2.mtmt.hu/api/journal/1427
Resolving m2.mtmt.hu (m2.mtmt.hu)... 

Új folyóirat:  ENGINEERING STRUCTURES 0141-0296 1873-7323


193.224.44.74
Connecting to m2.mtmt.hu (m2.mtmt.hu)|193.224.44.74|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/vnd.mtmt2-1.0+json]
Saving to: ‘journals/1427.json’

     0K ...                                                    36,6K=0,1s

2023-01-24 16:06:41 (36,6 KB/s) - ‘journals/1427.json’ saved [3749]

119it [09:09,  4.62s/it]


In [10]:
journal_categories = {}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'personal/{mtid}.json'):
            with open(f'personal/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:      
                        if "journal" in pub and pub["publishedYear"] >= 2012 and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            journal_categories[jour] = get_esi_category(jour).upper()
                except:
                    pass

119it [00:06, 18.46it/s]


In [13]:
max_year = 2023
pub_sum = {"tsz": [], "év": [], "szerző": [], "cím": [], "mtmt": [], "idézettség": [], "0.01%": [], "0.10%": [], "1.00%": [], "10.00%": [], "20.00%": [], "50.00%": []}
for x, a in tqdm(staff_flt.iterrows()):
    if not np.isnan(a["MTMT"]):
        mtid = int(a["MTMT"])
        if os.path.exists(f'personal/{mtid}.json'):
            with open(f'personal/{mtid}.json') as f:
                try:
                    js = json.load(f)
                    for pub in js:     
                        if "journal" in pub and "publishedYear" in pub and pub["publishedYear"] >= 2012 and pub["publishedYear"] <= max_year and check_wos(pub):
                            jour = pub["journal"]["link"].split("/")[-1]
                            if jour in journal_categories.keys() and "citationCount" in pub:
                                try:
                                    base = percentiles.loc[(journal_categories[jour], pub["publishedYear"]), :]
                                except KeyError:
                                    print(pub["journal"]["label"], jour, (journal_categories[jour], pub["publishedYear"]))
                                pub_sum["tsz"].append(a["Tanszék"])
                                pub_sum["év"].append(pub["publishedYear"])
                                pub_sum["szerző"].append(a["Név"])
                                pub_sum["cím"].append(pub["title"])
                                pub_sum["idézettség"].append(pub["citationCount"])
                                pub_sum["mtmt"].append(pub["mtid"])
                                r = (pub["citationCount"] >= base).astype(np.int32)
                                pub_sum["0.01%"].append(r["0.01%"])
                                pub_sum["0.10%"].append(r["0.10%"])
                                pub_sum["1.00%"].append(r["1.00%"])
                                pub_sum["10.00%"].append(r["10.00%"])
                                pub_sum["20.00%"].append(r["20.00%"])
                                pub_sum["50.00%"].append(r["50.00%"])
                except:
                    print(sys.exc_info()[0])
                    pass
pubs = pd.DataFrame(pub_sum).drop_duplicates("mtmt")

11it [00:00, 53.24it/s]

LETTERS IN MATHEMATICAL PHYSICS 0377-9017 1573-0530 3035 ('PHYSICS', 2023)


26it [00:02, 11.40it/s]

APPLIED MATHEMATICS AND COMPUTATION 0096-3003 1873-5649 454 ('MATHEMATICS', 2023)


47it [00:02, 26.08it/s]

...

JOURNAL OF MATHEMATICAL ANALYSIS AND APPLICATIONS 0022-247X 1096-0813 2634 ('MATHEMATICS', 2023)


92it [00:05, 27.80it/s]

EUROPEAN JOURNAL OF COMBINATORICS 0195-6698 1095-9971 1417130 ('MATHEMATICS', 2023)


119it [00:06, 19.75it/s]


JOURNAL OF THE AMERICAN MATHEMATICAL SOCIETY 0894-0347 1088-6834 13001 ('MATHEMATICS', 2023)


In [14]:
pubs[pubs["1.00%"]>0]

,tsz,év,szerző,cím,mtmt,idézettség,0.01%,0.10%,1.00%,10.00%,20.00%,50.00%
31,Sztochasztika,2019,Bárány Balázs,Hausdorff dimension of planar self-affine sets...,30603311,45,0,0,1,1,1,1
172,Algebra,2016,Hegedűs Pál,On the Discrete Logarithmic Minkowski Problem,3121562,69,0,0,1,1,1,1
211,Analízis,2022,Karátson János,Quasi-Newton variable preconditioning for nonl...,32034148,7,0,0,1,1,1,1
306,Differenciálegyenletek,2018,Kovács Mihály,Boundary conditions for fractional diffusion,3348787,45,0,0,1,1,1,1
431,Sztochasztika,2022,Molontay Roland,EASY-APP: An artificial intelligence model and...,32865751,9,0,0,1,1,1,1
679,Sztochasztika,2015,Vető Bálint,Height Fluctuations for the Stationary KPZ Equ...,2995258,101,0,0,1,1,1,1
724,Analízis,2018,Weiner Mihály,From vertex operator algebras to conformal net...,3327040,61,0,0,1,1,1,1


In [16]:
pubs.to_csv("percentille.csv", index=False, float_format='%.4g')

In [17]:
def score_person(mtid):
    if not os.path.exists(f'staff/{mtid}.json'):
        raise Exception("Nincs letöltve a publikációs lista!")
    pubs = json.load(open(f'staff/{mtid}.json'))
    score = score_publist([mtid], pubs)
    score["mtid"] = int(mtid)
    return score
    
def score_publist(mtid_list, pubs):
    score = {"pub_count": 0, "q_paper": 0, "q_n_paper": 0, "q_book": 0, "if": 0, "relif": 0, "ifcnt": 0, "if_norm": 0, "relif_norm": 0, "h": 0, "i": 0, "first_pub": 2100, "last_pub": 0}
    cit_list = []
    useShare = False
    ix = 0
    for pub in pubs:
        ix += 1
        if "error" in pub and pub["error"]!="VALIDATION_ERROR":
            continue
        if "category" not in pub or pub["category"]["label"]!="Tudományos":
            continue
        # compute share of the authos
        authors = 0
        share = len(mtid_list)/len(pub["authorships"]) if len(pub["authorships"])>0 else 0
        for a in pub["authorships"]:
            if "author" in a and a["author"]["mtid"] in mtid_list and a["authorTyped"]:
                authors += 1
        if authors < len(mtid_list):
            continue
        # compute number of pages
        plength = 0
        try:
            if "pageLength" in pub:
                plength = int(pub["pageLength"])
            elif "firstPage" in pub and "lastPage" in pub:
                plength = int(pub["lastPage"]) - int(pub["firstPage"]) + 1            
        except:
            pass
        # compute impact factor
        ifct, nrm = getif(pub)        
        # compute Q score
        if pub["otype"] == "JournalArticle" and "journal" in pub:
            if "fullPublication" in pub and pub["fullPublication"] and "reviewType" in pub["journal"] and pub["journal"]["reviewType"]=="REVIEWED" and "subType" in pub and \
                pub["subType"]["name"] in ["Szakcikk", "Összefoglaló cikk", "Konferenciaközlemény", "Rövid közlemény", "Sokszerzős vagy csoportos szerzőségű szakcikk"]:                
                if ifct > 0:
                    totalscr = max(0.6,ifct)
                elif "foreignEdition" in pub and pub["foreignEdition"]:
                    totalscr = 0.4
                else:
                    totalscr = 0.3
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share / nrm if nrm > 0 else totalscr * share
                if ifct > 0:
                    score["if"] += ifct
                    score["relif"] += totalscr * share
                    score["ifcnt"] += 1
                    score["if_norm"] += ifct / nrm if nrm > 0 else ifct
                    score["relif_norm"] += totalscr * share / nrm if nrm > 0 else totalscr * share
        elif "fullPublication" in pub and pub["fullPublication"] and \
            ((pub["type"]["label"]=="Könyvrészlet" and "subType" in pub and pub["subType"]["label"]=="Konferenciaközlemény (Könyvrészlet)") or pub["type"]["label"]=="Egyéb konferenciaközlemény"):                
            if plength >= 4:
                totalscr = 0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1
                score["q_paper"] += totalscr * share
                score["q_n_paper"] += totalscr * share
        elif pub["type"]["label"] == "Könyv" or \
            (pub["type"]["label"] == "Könyvrészlet" and "subType" in pub and pub["subType"]["label"] in ["Könyvfejezet (Könyvrészlet)", "Szaktanulmány (Könyvrészlet)"]):
            if plength >= 10:
                if plength >= 100:
                    totalscr = 2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 1
                else:
                    totalscr = (0.2 if "foreignLanguage" in pub and pub["foreignLanguage"] else 0.1) * math.floor(plength/10)
                score["q_book"] += totalscr * share
        # Process citations
        if "independentCitationCount" in pub:
            score["i"] += pub["independentCitationCount"]
            cit_list.append(pub["independentCitationCount"])
        if "publishedYear" in pub:
            score["last_pub"] = max(score["last_pub"], pub["publishedYear"])
            score["first_pub"] = min(score["first_pub"], pub["publishedYear"])
        score["pub_count"] += 1
    score["q"] = score["q_paper"] + np.minimum(score["q_book"], 3.0)
    score["qn"] = score["q_n_paper"] + np.minimum(score["q_book"], 3.0)
    # h-index
    cit_list.sort(reverse=True)
    while score["h"] < len(cit_list) and score["h"] < cit_list[score["h"]]:
        score["h"] += 1
        
    return score

In [19]:
ifdf = pd.read_csv('./pubgraph/ifdf_v8_2021.csv')
ifdf.loc[ifdf["if"].isnull(), "if"] = 0.0
maxIFYear = ifdf.year.max()
ifdf_e = ifdf.copy().set_index(["year", "eissn"]).sort_index()
ifdf_p = ifdf.copy().set_index(["year", "pissn"]).sort_index()

def getif(pub):
    if "journal" not in pub:
        return 0.0, 1.0
    year = min(pub["publishedYear"], maxIFYear)
    if "pIssn" in pub["journal"]:
        issn = pub["journal"]["pIssn"]
        if (year,issn) in ifdf_p.index:
            ifval = ifdf_p.loc[(year,issn),"if"]
            catif = ifdf_p.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    if "eIssn" in pub["journal"]:
        issn = pub["journal"]["eIssn"]
        if (year,issn) in ifdf_e.index:
            ifval = ifdf_e.loc[(year,issn),"if"]
            catif = ifdf_e.loc[(year,issn),"categoryMedianIf"]
            if len(ifval) == 1 and float(ifval) > 0.0:
                if len(catif) == 1 and float(catif) > 0.0:
                    return float(ifval), float(catif)
                else:
                    return float(ifval), float(ifval) # ha if-es az újság, de még nem elég régóta. nincs sok ilyen.
    return 0.0, 1.0

# a függvény visszaadja a folyóirat rangját (D1, Q1, Q2, ...)
def getrating(pub):
    if "ratings" not in pub:
        return ""
    for r in pub["ratings"]:
        if r["otype"] == "SjrRating" and "ranking" in r:
            return r["ranking"]
    return ""


In [20]:
dta = {"MTMT": [], "pubCount": [], "qScore": [], "qnScore": [], "ifCount": [], "ifScore": [], "relifScore": [], "ifNormScore": [], "relifNormScore": [], "citations": [], "hIndex": [], "firstPub": [], "lastPub": []}
for idx, prs in staff_flt.iterrows():
    scr = score_person(prs["MTMT"])
    dta["MTMT"].append(scr["mtid"])
    dta["pubCount"].append(scr["pub_count"])
    dta["qScore"].append(scr["q"])
    dta["qnScore"].append(scr["qn"])
    dta["ifCount"].append(scr["ifcnt"])
    dta["ifScore"].append(scr["if"])
    dta["relifScore"].append(scr["relif"])
    dta["ifNormScore"].append(scr["if_norm"])
    dta["relifNormScore"].append(scr["relif_norm"])
    dta["citations"].append(scr["i"])
    dta["hIndex"].append(scr["h"])
    dta["firstPub"].append(scr["first_pub"])
    dta["lastPub"].append(scr["last_pub"])
node_person = pd.DataFrame(dta)
node_person[node_person["pubCount"]>0].to_csv("node_person.csv", index=False, float_format='%.4g')

In [21]:
node_person

,MTMT,pubCount,qScore,qnScore,ifCount,ifScore,relifScore,ifNormScore,relifNormScore,citations,hIndex,firstPub,lastPub
0,10012522,23,9.162250,6.800885,10,10.856,8.162250,7.796477,5.800885,98,7,1998,2020
1,10041405,36,7.200000,16.093861,6,0.918,2.400000,4.281639,11.293861,42,3,1970,2009
2,10011729,33,15.860783,12.435711,26,39.425,15.727450,28.957170,12.302377,220,8,1997,2021
3,10042381,15,1.733333,1.733333,0,0.000,0.000000,0.000000,0.000000,3,1,2008,2017
4,10011669,31,7.139625,3.533433,10,21.269,5.916898,8.404271,2.310706,168,8,1985,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
115,10019824,20,8.937917,7.132186,15,15.574,8.287917,12.140872,6.482186,200,6,2006,2022
116,10042219,39,29.287500,14.404563,30,67.256,29.220833,31.425362,14.337897,144,7,2008,2023
117,10015273,33,25.605867,15.803946,30,53.344,25.372533,33.254772,15.570612,325,10,2001,2022
